# Process manager

The process manager allows the construction of sets of tasks to be run, which can be (but are not limited to) launch files or ROS nodes to bring up a system that may include an instance of Gazebo.
Each instance of the process manager can be set with a different `ROS_MASTER_URI` and `GAZEBO_MASTER_URI`, so that multiple `roscore` and `gazebo` instances can be initialized on the same host at the same time.

## Starting a process manager with default parameters

In [1]:
from pcg_gazebo.task_manager import ProcessManager
# A process manager can be started by itself with an empty list of tasks
process_manager = ProcessManager()
# At this point neither Gazebo or roscore are running
# When no arguments are given, the default hostname and port
# arguments are going to be used.
print('ROS network configuration:')
print(process_manager.ros_config)

ROS network configuration:
ROS_MASTER_URI=http://localhost:11311, GAZEBO_MASTER_URI=http://localhost:11345


In [2]:
# At first, no tasks are available in the tasks list
print(process_manager.get_task_list())

[]


In [3]:
print('Check all the process stages available')
print(process_manager.stages)

Check all the process stages available
OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f59f7d31f28>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f59f7d31f60>)])


In [4]:
# You can start roscore by calling the method below
process_manager.create_ros_core_task()

2020-09-11 15:09:52,567 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


True

In [5]:
# Now the roscore task can be found in the list
print(process_manager.get_task_list())

['roscore']


In [6]:
# The task can be started using the method below
# Running individual tasks will not use the stage order 
process_manager.run_task('roscore')

2020-09-11 15:10:46,785 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 15:10:47,040 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


In [7]:
# Check if the task is running
print('Is task running? {}'.format(process_manager.is_task_running('roscore')))

Is task running? True


In [8]:
# Creating an RViz task with a timeout
# The required flag, like the flag for ROS nodes, says that once this task dies, 
# all other tasks must be killed.
# The process timeout starts a timer and will kill the task in the amount of seconds
# given by process_timeout
# IMPORTANT: process_timeout is based on the machine clock, not the simulation clock
# process_timeout=None means that the process will run without a timeout
process_manager.create_rviz_task(required=True, process_timeout=10)
print(process_manager.get_task_list())

['rviz', 'roscore']


In [9]:
# After 10 seconds all tasks will be killed along with rviz
process_manager.run_task('rviz')
process_manager.wait()


2020-09-11 15:13:25,572 | WARNING | task | Task <rviz> - Killing process tree...
2020-09-11 15:13:25,604 | WARNING | task | Sending SIGINT to child process id=26127
2020-09-11 15:13:25,608 | WARNING | task | Child process 26127 still running
2020-09-11 15:13:25,610 | WARNING | task | Sending SIGINT to child process id=26126
2020-09-11 15:13:25,612 | WARNING | task | Child process 26126 still running
2020-09-11 15:13:25,998 | WARNING | task | Process 26126 already terminated
2020-09-11 15:13:25,999 | WARNING | task | Process 26127 already terminated
2020-09-11 15:13:25,999 | WARNING | task | Kill processes=
	 - Gone=[psutil.Process(pid=26126, status='terminated', exitcode=<Negsignal.SIGINT: -2>, started='15:13:14'), psutil.Process(pid=26127, status='terminated', started='15:13:14')]
	 - Alive[]
2020-09-11 15:13:26,000 | WARNING | process_manager | Required task <rviz> has been terminated, killing all tasks
2020-09-11 15:13:26,000 | WARNING | task | Task <roscore> - Killing process tree.

In [10]:
process_manager.kill_all_tasks()
del process_manager

2020-09-11 15:13:30,981 | WARNING | task | Task <rviz> - Process object is invalid
2020-09-11 15:13:30,982 | WARNING | task | Task <roscore> - Process object is invalid


## Running Gazebo

Similar to RViz, Gazebo can also be started. The same process will be repeated to start Gazebo with the empty world scenario.

In [11]:
# At first, no tasks are available in the tasks list
# A process manager can be started by itself with an empty list of tasks
# When ros_port and/or gazebo_port is given as None, a random port will be chosen
process_manager = ProcessManager(ros_port=None, gazebo_port=None)
print('ROS network configuration:')
print(process_manager.ros_config)

ROS network configuration:
ROS_MASTER_URI=http://localhost:15048, GAZEBO_MASTER_URI=http://localhost:27125


In [12]:
# A Gazebo task can also be started with a process timeout
process_manager.create_gazebo_empty_world_task(required=True, process_timeout=10)
print('Check all tasks available')
print(process_manager.get_task_list())
print('Check all the process stages available')
print(process_manager.stages)

2020-09-11 15:13:53,241 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
Check all tasks available
['gazebo']
Check all the process stages available
OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f5a586bb0f0>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f59f7ce16a0>), ('gazebo', <pcg_gazebo.task_manager.stage.Stage object at 0x7f5a586bb128>)])


In [13]:
process_manager.run_all_tasks()
# A roscore stage should be automatically added to the process manager list
print(process_manager.stages)
process_manager.wait()


OrderedDict([('roscore', <pcg_gazebo.task_manager.stage.Stage object at 0x7f5a586bb0f0>), ('pre-simulation', <pcg_gazebo.task_manager.stage.Stage object at 0x7f59f7ce16a0>), ('gazebo', <pcg_gazebo.task_manager.stage.Stage object at 0x7f5a586bb128>)])
2020-09-11 15:14:04,446 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 15:14:04,679 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 15:14:04,931 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 15:14:15,694 | WARNING | task | Task <gazebo> - Killing process tree...
2020-09-11 15:14:15,737 | WARNING | task | Sending SIGINT to child process id=26370
2020-09-11 15:14:15,738 | WARNING | task | Child process 26370 still running
2020-09-11 15:14:15,739 | WARNING | task | Sending SIGINT to child process id=2639

In [14]:
process_manager.kill_all_tasks()
del process_manager


2020-09-11 15:14:22,073 | WARNING | task | Task <roscore> - Process object is invalid
2020-09-11 15:14:22,074 | WARNING | task | Task <gazebo> - Process object is invalid


In [15]:
# But a task that contains a Gazebo instance can also be started with a simulation timeout
# meaning that the process will be killed only when the simulation time reaches a timeout
# At first, no tasks are available in the tasks list
# A process manager can be started by itself with an empty list of tasks
# When ros_port and/or gazebo_port is given as None, a random port will be chosen
process_manager = ProcessManager(ros_port=None, gazebo_port=None)
print('ROS network configuration:')
print(process_manager.ros_config)

process_manager.create_gazebo_empty_world_task(required=True, simulation_timeout=10)
process_manager.run_all_tasks()
process_manager.wait()

ROS network configuration:
ROS_MASTER_URI=http://localhost:17899, GAZEBO_MASTER_URI=http://localhost:26148


In [16]:
print('Is task running? {}'.format(process_manager.is_task_running('gazebo')))

Is task running? False


In [17]:
# End the simulation by killing the Gazebo task
process_manager.kill_all_tasks()